# Total Seigniorage Calculation

How much seigniorage generated in case of `one commit` and `every commit` in same blocks

`p1 case - every commit in blocks` 
```
blocks  :  1...2...3..  ...2425845...2425846
commits : c1, c2, c3,.. ..c2425845, c2425846
```
`p2 case - one commit in blocks` 
```
blocks  :  1...2...3..  ...2425845...2425846
commits :                                 c1
```

In [1]:
from typing import List
from random import random, randrange
from dataclasses import asdict

import names
from powerton import User, Operator, Player, Powerton, OMD

import pandas as pd
pd.options.display.float_format = '{:.18f}'.format

p1 = Powerton()
p2 = Powerton()

# set to No winning chance
p1.AIRDROP_CHANCE = -1
p1.WINNING_CHANCE = -1

p2.AIRDROP_CHANCE = -1
p2.WINNING_CHANCE = -1

In [2]:
p1.SEIG_PER_BLOCK

78.32319116712273

In [3]:
p1.add_user("Operator", OMD)
p1.add_user("User", p1.INITIAL_SUPPLY - OMD)
p1.add_operator("Operator")
p1.delegate("User", "Operator", 999000000)

In [4]:
asdict(p1.Operators["Operator"])

{'name': 'Operator',
 'balance': 0.0,
 'delegated': 1000000.0,
 'delegators': [],
 'supported_power': 0,
 'power_supporters': {},
 'delegatees': ['Operator', 'User'],
 'delegated_balances': {'Operator': 1000000, 'User': 999000000}}

In [5]:
p1._update_ton()

In [6]:
# Override TON balance
# Only `User` has all TON
p1.Users['User'].delegated = p1.INITIAL_SUPPLY
p1.Operators['Operator'].delegated_balances['User'] = p1.INITIAL_SUPPLY
p1.Operators['Operator'].delegated = 0
p1.Operators['Operator'].delegated_balances['Operator'] = 0

In [7]:
asdict(p1.Users['User'])

{'name': 'User', 'balance': 0.0, 'delegated': 1000000000, 'delegators': []}

In [8]:
asdict(p1.Operators['Operator'])

{'name': 'Operator',
 'balance': 0.0,
 'delegated': 0,
 'delegators': [],
 'supported_power': 0,
 'power_supporters': {},
 'delegatees': ['Operator', 'User'],
 'delegated_balances': {'Operator': 0, 'User': 1000000000}}

In [9]:
p1_o = asdict(p1.Operators["Operator"])
p1_u = asdict(p1.Users["User"])

p1.Total_TON_supply == p1_o['delegated'] + p1_u['delegated']

True

In [10]:
p1.Staking_ratio

1.0

In [11]:
for i in range(p1.BLOCKS_YEAR): # 1 Year estimated block
    p1.commit("Operator")
    p1.wrap()

p2._update_ton()

In [12]:
p1.Staking_ratio 

1.0

In [13]:
asdict(p1.Operators["Operator"])

{'name': 'Operator',
 'balance': 0.0,
 'delegated': 0.0,
 'delegators': [],
 'supported_power': 0,
 'power_supporters': {},
 'delegatees': ['Operator', 'User'],
 'delegated_balances': {'Operator': 0.0, 'User': 1189999921.673901}}

In [14]:
p2.add_user("Operator", OMD)
p2.add_user("User", p2.INITIAL_SUPPLY - OMD)
p2.add_operator("Operator")
p2.delegate("User", "Operator", 999000000)

In [15]:
p2._update_ton()

In [16]:
# Override Supply
p2.Users['User'].delegated = p2.INITIAL_SUPPLY
p2.Operators['Operator'].delegated_balances['User'] = p2.INITIAL_SUPPLY
p2.Operators['Operator'].delegated = 0
p2.Operators['Operator'].delegated_balances['Operator'] = 0

In [17]:
p2.Users

{'User': User(name='User', balance=0.0, delegated=1000000000)}

In [18]:
p2.Operators

{'Operator': Operator(name='Operator', balance=0.0, delegated=0, supported_power=0, power_supporters={}, delegatees=['Operator', 'User'], delegated_balances={'Operator': 0, 'User': 1000000000})}

In [19]:
p2_o = asdict(p2.Operators["Operator"])
p2_u = asdict(p2.Users["User"])

In [20]:
p2_o

{'name': 'Operator',
 'balance': 0.0,
 'delegated': 0,
 'delegators': [],
 'supported_power': 0,
 'power_supporters': {},
 'delegatees': ['Operator', 'User'],
 'delegated_balances': {'Operator': 0, 'User': 1000000000}}

In [21]:
p2.Total_TON_supply == p2_o['delegated'] + p2_u['delegated']

True

In [22]:
p2.Staking_ratio

1.0

In [23]:
p2._update_ton()
p2.wrap(p2.BLOCKS_YEAR)
p2.commit("Operator")

(False, False)

In [24]:
p2.Staking_ratio 

1.0

In [25]:
asdict(p2.Operators["Operator"])

{'name': 'Operator',
 'balance': 0.0,
 'delegated': 0.0,
 'delegators': [],
 'supported_power': 0,
 'power_supporters': {},
 'delegatees': ['Operator', 'User'],
 'delegated_balances': {'Operator': 0.0, 'User': 1190000000.0}}

In [26]:
compare_list = ["Current_block", 
                "Last_committed",
                "Total_TON_supply",
                "Total_TON_staked",
                "Staking_ratio",
                "Prize_pool",
                "Airdrop_pool"]

In [27]:
data = {}
for col in compare_list:
    _p1 = eval("p1.{}".format(col))
    _p2 = eval("p2.{}".format(col))
    _diff = _p1 - _p2
    data[col] = {"every_commit": _p1, "one_commit" : _p2}

df = pd.DataFrame(data).T
df['diff'] = df['every_commit'] - df['one_commit']

df

,every_commit,one_commit,diff
Current_block,2425846.000000000000000000,2425846.000000000000000000,0.000000000000000000
Last_committed,2425845.000000000000000000,2425846.000000000000000000,-1.000000000000000000
Total_TON_supply,1189999921.673901081085205078,1190000000.000000000000000000,-78.326098918914794922
Total_TON_staked,1189999921.673901081085205078,1190000000.000000000000000000,-78.326098918914794922
Staking_ratio,1.000000000000000000,1.000000000000000000,0.000000000000000000
Prize_pool,0.000000000000000000,0.000000000000000000,0.000000000000000000
Airdrop_pool,0.000000000000000000,0.000000000000000000,0.000000000000000000


## Seigniorage Calculation Two Staker

`User` and `Operator` staked 50:50 of Total Staked TON

In [28]:
p1 = Powerton()
p2 = Powerton()

# set to No winning chance
p1.AIRDROP_CHANCE = -1
p1.WINNING_CHANCE = -1

p2.AIRDROP_CHANCE = -1
p2.WINNING_CHANCE = -1

In [29]:
# Add Participant
p1.add_user("Operator", OMD)
p1.add_user("User", p1.INITIAL_SUPPLY - OMD)
p1.add_operator("Operator")
p1.delegate("User", "Operator", 999000000)

In [30]:
# Override TON balance
# Only `User` has all TON
p1.Users['User'].delegated = p1.INITIAL_SUPPLY / 2
p1.Operators['Operator'].delegated_balances['User'] = p1.INITIAL_SUPPLY / 2
p1.Operators['Operator'].delegated = p1.INITIAL_SUPPLY / 2
p1.Operators['Operator'].delegated_balances['Operator'] = p1.INITIAL_SUPPLY / 2

In [31]:
asdict(p1.Users['User'])

{'name': 'User', 'balance': 0.0, 'delegated': 500000000.0, 'delegators': []}

In [32]:
asdict(p1.Operators['Operator'])

{'name': 'Operator',
 'balance': 0.0,
 'delegated': 500000000.0,
 'delegators': [],
 'supported_power': 0,
 'power_supporters': {},
 'delegatees': ['Operator', 'User'],
 'delegated_balances': {'Operator': 500000000.0, 'User': 500000000.0}}

In [33]:
p1_o = asdict(p1.Operators["Operator"])
p1_u = asdict(p1.Users["User"])

p1.Total_TON_supply == p1_o['delegated'] + p1_u['delegated']

p1._update_ton()

In [34]:
for i in range(p1.BLOCKS_YEAR): # 1 Year estimated block
    p1.commit("Operator")
    p1.wrap()

p2._update_ton()

In [35]:
p1.Staking_ratio

1.0

In [36]:
p2.add_user("Operator", OMD)
p2.add_user("User", p2.INITIAL_SUPPLY - OMD)
p2.add_operator("Operator")
p2.delegate("User", "Operator", 999000000)

In [37]:
# Override TON balance
# Only `User` has all TON
p2.Users['User'].delegated = p2.INITIAL_SUPPLY / 2
p2.Operators['Operator'].delegated_balances['User'] = p1.INITIAL_SUPPLY / 2
p2.Operators['Operator'].delegated = p2.INITIAL_SUPPLY / 2
p2.Operators['Operator'].delegated_balances['Operator'] = p1.INITIAL_SUPPLY / 2

In [38]:
p2._update_ton()
p2.wrap(p2.BLOCKS_YEAR)
p2.commit("Operator")

(False, False)

In [39]:
asdict(p2.Operators["Operator"])

{'name': 'Operator',
 'balance': 0.0,
 'delegated': 595000000.0,
 'delegators': [],
 'supported_power': 0,
 'power_supporters': {},
 'delegatees': ['Operator', 'User'],
 'delegated_balances': {'Operator': 595000000.0, 'User': 595000000.0}}

In [40]:
data = {}
for col in compare_list:
    _p1 = eval("p1.{}".format(col))
    _p2 = eval("p2.{}".format(col))
    _diff = _p1 - _p2
    data[col] = {"every_commit": _p1, "one_commit" : _p2}

df = pd.DataFrame(data).T
df['diff'] = df['every_commit'] - df['one_commit']

df

,every_commit,one_commit,diff
Current_block,2425846.000000000000000000,2425846.000000000000000000,0.000000000000000000
Last_committed,2425845.000000000000000000,2425846.000000000000000000,-1.000000000000000000
Total_TON_supply,1189999921.673901081085205078,1190000000.000000000000000000,-78.326098918914794922
Total_TON_staked,1189999921.673901081085205078,1190000000.000000000000000000,-78.326098918914794922
Staking_ratio,1.000000000000000000,1.000000000000000000,0.000000000000000000
Prize_pool,0.000000000000000000,0.000000000000000000,0.000000000000000000
Airdrop_pool,0.000000000000000000,0.000000000000000000,0.000000000000000000
